In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.W1 = nn.Linear(self.hidden_dim, self.hidden_dim, bias=False)
        self.W2 = nn.Linear(self.hidden_dim, self.hidden_dim, bias=False)
        self.V = nn.Linear(self.hidden_dim, 1, bias=False)
        self.tanh = nn.Tanh()

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight.data)

    def forward(self, enc_outputs, dec_output, mask):
        w1_e = self.W1(enc_outputs)
        w2_d = self.W2(dec_output)
        tanh_output = self.tanh(w1_e + w2_d)
        v_dot_tanh = self.V(tanh_output).squeeze(2)
        v_dot_tanh += mask
        attention_weights = F.softmax(v_dot_tanh, dim=1)
        return attention_weights

# Encoder Network
class Encoder(nn.Module):
    def __init__(self, hidden_dim, input_dim=2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.cell = nn.GRU(self.input_dim, self.hidden_dim, 1, batch_first=True)

    def forward(self, input):
        enc_output, enc_hidden_state = self.cell(input)
        return enc_output, enc_hidden_state

# Decoder Network
class Decoder(nn.Module):
    def __init__(self, hidden_dim, input_dim=2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.cell = nn.GRU(self.input_dim, self.hidden_dim, 1, batch_first=True)
        self.attention_layer = Attention(self.hidden_dim)

    def forward(self, input, enc_output, hidden_state, pointer, mask):
        idx = pointer.repeat(1, 2).unsqueeze(1)
        dec_output, dec_hidden = self.cell(input.gather(1, idx), hidden_state)
        attention_weights = self.attention_layer(enc_output, dec_output, mask)
        return attention_weights, dec_hidden

ModuleNotFoundError: No module named 'torch'

In [ ]:
class Critic(nn.Module):
    def __init__(self, hidden_dim, input_dim=2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim

        self.encoder = Encoder(self.hidden_dim)
        self.decoder_1 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.decoder_2 = nn.Linear(self.hidden_dim, 1)
        self.relu = nn.ReLU()

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight.data)

    def forward(self, input):
        enc_output, enc_hidden_state = self.encoder(input)
        dec_hidden_state = self.decoder_1(enc_hidden_state)
        dec_hidden_state = self.relu(dec_hidden_state)
        dec_output = self.decoder_2(dec_hidden_state)
        return dec_output.squeeze(0)


In [ ]:
class PCTSPEnvironment:
    def __init__(self, num_cities, max_reward, max_distance):
        self.num_cities = num_cities
        self.max_reward = max_reward
        self.max_distance = max_distance
        self.reset()

    def reset(self):
        self.city_coordinates = torch.rand((self.num_cities, 2))
        self.city_rewards = torch.randint(1, self.max_reward, (self.num_cities,))
        self.distance_travelled = 0
        self.current_city = 0
        self.visited_cities = set([0])
        print("Random Data for Current Episode:")
        print("City Coordinates:\n", self.city_coordinates)
        print("City Rewards:\n", self.city_rewards)
        print("distance_travelled:\n", self.distance_travelled)

    def get_state(self):
        return self.city_coordinates

    def step(self, action):
        next_city = action.item()
        reward = 0
        done = False

        distance_to_next_city = torch.norm(self.city_coordinates[self.current_city] - self.city_coordinates[next_city])

        if next_city not in self.visited_cities and self.distance_travelled + distance_to_next_city <= self.max_distance:
            reward += self.city_rewards[next_city]
            self.distance_travelled += distance_to_next_city
            self.visited_cities.add(next_city)
            self.current_city = next_city
        else:
            done = True

        return self.get_state(), reward, done


In [ ]:
# PtrNet Network
class PtrNet(nn.Module):
    def __init__(self, hidden_dim, input_dim=2, deterministic=False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.deterministic = deterministic
        self.encoder = Encoder(self.hidden_dim)
        self.decoder = Decoder(self.hidden_dim)
    
    def get_length(self, input, path_indices):
        """
        Calculate the total length of the path.
        Args:
        - input: Tensor of city coordinates.
        - path_indices: Indices of the cities in the path.
        Returns:
        - Total length of the path.
        """
        batch_size, _, _ = input.size()
        path_length = path_indices.size(1)  # 실제 경로 길이
        length = 0

        path_indices = path_indices.long()

        for i in range(path_length - 1):  # 실제 경로 길이를 기준으로 반복
            current_city = path_indices[:, i]
            next_city = path_indices[:, i + 1]
            distance = torch.norm(input[torch.arange(batch_size), current_city] - input[torch.arange(batch_size), next_city], dim=1)
            length += distance

        # 마지막 도시에서 첫 번째 도시로 돌아가는 거리 계산
        last_city = path_indices[:, -1]
        first_city = path_indices[:, 0]
        distance = torch.norm(input[torch.arange(batch_size), last_city] - input[torch.arange(batch_size), first_city], dim=1)
        length += distance
        print("step length: ",length)

        return length
    
    def forward(self, input, city_rewards, max_distance):
        batch_size = input.size(0)
        seq_len = input.size(1)

        # 초기화
        probs = torch.zeros(batch_size, 1, device=input.device)
        pointers = torch.zeros(batch_size, 0, dtype=torch.long, device=input.device)
        mask = torch.zeros(batch_size, seq_len, device=input.device)
        distance_travelled = torch.zeros(batch_size, 1, device=input.device)
        total_reward = torch.zeros(batch_size, 1, device=input.device, requires_grad=True)
        prev_pointer = torch.zeros(batch_size, 1, dtype=torch.long, device=input.device)
        pointer = torch.zeros(batch_size, 1, dtype=torch.long, device=input.device)  # 추가된 초기화

        enc_output, enc_hidden_state = self.encoder(input)
        mask = self.update_mask(mask, pointer)  # 이제 pointer가 정의됨
        dec_hidden_state = enc_hidden_state

        for i in range(seq_len - 1):
            attention_weights, dec_hidden_state = self.decoder(input, enc_output, dec_hidden_state, pointer, mask)
            # ... (나머지 코드)

            if self.deterministic:
                prob, pointer = torch.max(attention_weights, dim=1)
                pointer = pointer.unsqueeze(1)  # 포인터 차원 변경
            else:
                pointer = attention_weights.multinomial(1, replacement=True)
                prob = torch.gather(attention_weights, 1, pointer)

            probs += torch.log(prob)
            pointers = torch.cat([pointers, pointer], dim=1)
            mask = self.update_mask(mask, pointer)

            # Calculate distance and reward
            distance = torch.norm(input[torch.arange(batch_size), prev_pointer.squeeze(-1)] - input[torch.arange(batch_size), pointer.squeeze(-1)], dim=1).unsqueeze(1)
            distance_travelled += distance
            
            # Assuming city_rewards is of shape [batch_size, num_cities]
            
            pointer_for_reward = pointer.squeeze(-1)  # [batch_size, 1] -> [batch_size]
            pointer_for_reward = pointer_for_reward.unsqueeze(1)  # Reshape to [batch_size, 1] for gather

            if city_rewards.dim() == 1:
                # If city_rewards is a 1D tensor, unsqueeze it to match dimensions for gather
                reward = torch.gather(city_rewards.unsqueeze(0), 1, pointer_for_reward)
            elif city_rewards.dim() == 2:
                # If city_rewards is a 2D tensor
                reward = torch.gather(city_rewards, 1, pointer_for_reward)
            else:
                raise ValueError("Unsupported shape for city_rewards")

            total_reward = total_reward + reward.squeeze(1)

            prev_pointer = pointer

            # Check if distance exceeds max distance
            if torch.any(distance_travelled > max_distance):
                break

        return total_reward, pointers

    def update_mask(self, mask, pointer):
        for batch, i in enumerate(pointer):
            mask[batch, i] = float('-inf')
        return mask
    

In [ ]:
# Training Loop
num_cities = 100
max_reward = 100
max_distance = 10
hidden_dim = 256
lr = 0.0001

env = PCTSPEnvironment(num_cities, max_reward, max_distance)
dataset = TensorDataset(env.get_state().unsqueeze(0))
dataloader = DataLoader(dataset, batch_size=64)

model = PtrNet(hidden_dim=hidden_dim)
critic = Critic(hidden_dim=hidden_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer_critic = torch.optim.Adam(critic.parameters(), lr=lr)

# Training Loop
# ...

for epoch in range(1000):
    for i, s_i in tqdm(enumerate(dataloader)):
        s_i = s_i[0]
        city_rewards = env.city_rewards

        # 모델을 통한 예측
        total_reward, pi = model(s_i, city_rewards, max_distance)
        L = model.get_length(s_i, pi)
        b = critic(s_i)

        # 첫 번째 손실 계산
        loss = -total_reward.mean()  # 보상 최대화
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        # 두 번째 손실 계산
        loss_critic = F.mse_loss(L.squeeze(), b.squeeze())  # 차원 일치를 위해 squeeze 사용
        optimizer_critic.zero_grad()
        loss_critic.backward()  # 여기서는 retain_graph=True 필요 없음
        optimizer_critic.step()

        if (i + 1) % 1 == 0:
            print(f"Epoch: {epoch + 1}, Step: {i + 1}, Reward: {total_reward.mean().item()}, Loss: {loss.item()}")

print("Training completed.")



Random Data for Current Episode:
City Coordinates:
 tensor([[0.3624, 0.7666],
        [0.1314, 0.2474],
        [0.4037, 0.3024],
        [0.8362, 0.7349],
        [0.0964, 0.0315],
        [0.6602, 0.2739],
        [0.0802, 0.7884],
        [0.5368, 0.1451],
        [0.4808, 0.4080],
        [0.4113, 0.6874],
        [0.0349, 0.8853],
        [0.1420, 0.8162],
        [0.3445, 0.5759],
        [0.2930, 0.1449],
        [0.4886, 0.9847],
        [0.0458, 0.0929],
        [0.5563, 0.5625],
        [0.8495, 0.8150],
        [0.9506, 0.1575],
        [0.9018, 0.7592],
        [0.0554, 0.1295],
        [0.2693, 0.1970],
        [0.3492, 0.2665],
        [0.7836, 0.8150],
        [0.9080, 0.0599],
        [0.0060, 0.6412],
        [0.5082, 0.0822],
        [0.2194, 0.9191],
        [0.9887, 0.0479],
        [0.2015, 0.6008],
        [0.2692, 0.5937],
        [0.8079, 0.7084],
        [0.0437, 0.2660],
        [0.0924, 0.2377],
        [0.1883, 0.7958],
        [0.1012, 0.6955],
        [0.1

1it [00:00,  8.42it/s]


step length:  tensor([10.6025])
Epoch: 1, Step: 1, Reward: 872.0, Loss: -872.0


1it [00:00, 18.80it/s]


step length:  tensor([9.8811])
Epoch: 2, Step: 1, Reward: 1037.0, Loss: -1037.0


...

tensor([10.3775])
Epoch: 999, Step: 1, Reward: 946.0, Loss: -946.0


0it [00:00, ?it/s]

step length: 

1it [00:00, 14.17it/s]

 tensor([10.1065])
Epoch: 1000, Step: 1, Reward: 1031.0, Loss: -1031.0
Training completed.


In [ ]:
# 테스트
import time

num_cities_test = 100  # 테스트할 도시 수
max_reward_test = 100  # 최대 보상 값
max_distance_test = 10  # 최대 이동 가능 거리
hidden_dim_test = 256  # 히든 디멘션

# 새로운 환경 생성
env_test = PCTSPEnvironment(num_cities_test, max_reward_test, max_distance_test)

# 랜덤 좌표 생성
test_coordinates = torch.rand((num_cities_test, 2))

# 모델과 데이터셋 준비
test_dataset = TensorDataset(test_coordinates.unsqueeze(0))
test_dataloader = DataLoader(test_dataset, batch_size=1)

# 모델과 크리틱 생성
test_model = PtrNet(hidden_dim=hidden_dim_test)
test_critic = Critic(hidden_dim=hidden_dim_test)

# 모델과 크리틱의 상태를 CPU로 이동
test_model.cpu()
test_critic.cpu()

# 모델 및 크리틱의 파라미터 불러오기
test_model.load_state_dict(model.state_dict())
test_critic.load_state_dict(critic.state_dict())

# 모델 및 크리틱을 평가 모드로 설정
test_model.eval()
test_critic.eval()

# 테스트 루프
total_distances = []
total_rewards = []
start_time = time.time()  # 시작 시간 기록

for i, s_i_test in tqdm(enumerate(test_dataloader)):
    s_i_test = s_i_test[0]

    # 모델을 통한 예측
    total_reward_test, pi_test = test_model(s_i_test, env_test.city_rewards, max_distance_test)
    total_distance_test = test_model.get_length(s_i_test, pi_test)

    # 결과 기록
    total_distances.append(total_distance_test.item())
    total_rewards.append(total_reward_test.item())

end_time = time.time()  # 종료 시간 기록
calculation_time = end_time - start_time  # 계산에 걸린 시간 계산

# 결과 출력
print(f"Total distance of the path: {sum(total_distances)}")
print(f"Total reward collected: {sum(total_rewards)}")
print(f"Calculation Time: {calculation_time} seconds")

Random Data for Current Episode:
City Coordinates:
 tensor([[4.1113e-02, 2.7530e-01],
        [3.3297e-01, 9.0903e-01],
        [1.3806e-01, 8.5710e-01],
        [6.0656e-01, 3.1280e-01],
        [1.2300e-01, 8.9359e-01],
        [9.7994e-01, 3.3151e-01],
        [6.0642e-01, 3.6744e-02],
        [6.8014e-01, 6.8255e-01],
        [3.1067e-01, 7.9592e-01],
        [1.7952e-01, 4.6431e-01],
        [3.5741e-01, 8.0906e-01],
        [9.1592e-01, 2.8179e-01],
        [1.3437e-02, 7.5215e-01],
        [4.4005e-01, 2.4348e-01],
        [2.2654e-02, 4.2046e-02],
        [9.5371e-01, 6.5710e-01],
        [2.8128e-01, 3.2654e-01],
        [4.5201e-01, 9.5240e-01],
        [9.2409e-01, 5.5540e-01],
        [6.6416e-01, 6.7016e-01],
        [9.0091e-01, 8.6161e-01],
        [5.2090e-01, 2.2549e-01],
        [6.2680e-01, 5.2165e-01],
        [5.1865e-01, 3.6116e-01],
        [3.1807e-01, 8.7791e-01],
        [5.4913e-01, 1.6708e-02],
        [9.4732e-01, 6.6145e-01],
        [6.2318e-01, 8.5188e-0

0it [00:00, ?it/s]

1it [00:00, 66.63it/s]

step length:  tensor([9.8192])
Total distance of the path: 9.819174766540527
Total reward collected: 1176.0
Calculation Time: 0.017009258270263672 seconds


In [ ]:
values = [
    40, 55, 60, 76, 42, 36, 9, 2, 51, 23, 33, 78, 27, 41, 84, 51, 6, 76,
    38, 57, 71, 32, 98, 16, 99, 31, 46, 52, 56, 10, 2, 82, 37, 44, 23, 12,
    33, 86, 57, 58, 22, 85, 38, 34, 64, 42, 89, 50, 54, 77, 95, 12, 90, 11,
    52, 87, 47, 93, 71, 69, 3, 86, 21, 9, 86, 29, 65, 80, 11, 80, 61, 22,
    38, 93, 77, 91, 26, 7, 39, 2, 46, 16, 66, 54, 92, 4, 32, 84, 80, 73,
    41, 56, 45, 84, 20, 52, 61, 30, 33, 3
]

total_sum = sum(values)
print("리스트의 총합:", total_sum)


리스트의 총합: 4910
